# Download the Dataset from Kyaggle


In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle


In [ ]:
!kaggle datasets download -d gunavenkatdoddi/eye-diseases-classification


 99% 728M/736M [00:06<00:00, 38.4MB/s]
100% 736M/736M [00:06<00:00, 110MB/s] 


In [ ]:
!unzip /content/eye-diseases-classification.zip

Archive:  /content/eye-diseases-classification.zip
  inflating: dataset/cataract/0_left.jpg  
  inflating: dataset/cataract/103_left.jpg  
  inflating: dataset/cataract/1062_right.jpg  
  inflating: dataset/cataract/1083_left.jpg  
  inflating: dataset/cataract/1084_right.jpg  
  inflating: dataset/cataract/1102_left.jpg  
  inflating: dataset/cataract/1102_right.jpg  
  inflating: dataset/cataract/1115_left.jpg  
  inflating: dataset/cataract/1126_right.jpg  
  inflating: dataset/cataract/112_right.jpg  
  inflating: dataset/cataract/1144_left.jpg  
  inflating: dataset/cataract/1144_right.jpg  
  inflating: dataset/cataract/1164_left.jpg  
  inflating: dataset/cataract/1167_right.jpg  
  inflating: dataset/cataract/119_left.jpg  
  inflating: dataset/cataract/1285_left.jpg  
  inflating: dataset/cataract/1415_left.jpg  
  inflating: dataset/cataract/1415_right.jpg  
  inflating: dataset/cataract/1435_left.jpg  
  inflating: dataset/cataract/1444_left.jpg  
  inflating: dataset/catara

#VGG19

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam


In [ ]:
#split the dataset into test and train
!pip install split-folders
import splitfolders
splitfolders.ratio('/content/dataset',output='data',seed=1337,ratio=(0.8,0.2))

Copying files: 4217 files [00:02, 1648.10 files/s]


In [ ]:
# Define the image dimensions and batch size
image_size = (224,224)
batch_size = 64

##Data Preprocessing

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

valid_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
train_data = train_datagen.flow_from_directory(
    directory='/content/data/train',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
)

validation_data = valid_datagen.flow_from_directory(
    directory='/content/data/val',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
  )

Found 3372 images belonging to 4 classes.
Found 845 images belonging to 4 classes.


## Model Building

In [ ]:
vgg = VGG19(
    include_top=False,
    weights="imagenet",
    input_shape=(240,240, 3),
)

for layer in vgg.layers:
    layer.trainable = False

# Add custom layers
x = vgg.output
x = Flatten()(x)

x = BatchNormalization()(x)
x= Dense(512, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # 4 categories

# Create new model
model = Model(inputs=vgg.input, outputs=predictions)




80134624/80134624 [==============================] - 4s 0us/step


In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 240, 240, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 240, 240, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 120, 120, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 120, 120, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 120, 120, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 60, 60, 128)       0   

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# ModelCheckpoint
model_checkpoint = ModelCheckpoint(
    'vgg19best_model.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

In [ ]:
#Train the model
model.fit(train_data,
                 validation_data=validation_data,
                 epochs=20,
                 batch_size=100,
                 steps_per_epoch=len(train_data),
                 validation_steps=len(validation_data),
                 #callbacks=[model_checkpoint]
                 )


Epoch 1/20
53/53 [==============================] - 85s 1s/step - loss: 4.5195 - accuracy: 0.6862 - val_loss: 2.9883 - val_accuracy: 0.4982
Epoch 2/20
53/53 [==============================] - 67s 1s/step - loss: 2.3846 - accuracy: 0.7714 - val_loss: 0.7659 - val_accuracy: 0.7101
Epoch 3/20
53/53 [==============================] - 67s 1s/step - loss: 0.7452 - accuracy: 0.7977 - val_loss: 0.9504 - val_accuracy: 0.5586
Epoch 4/20
53/53 [==============================] - 67s 1s/step - loss: 0.4765 - accuracy: 0.8277 - val_loss: 0.7224 - val_accuracy: 0.6757
Epoch 5/20
53/53 [==============================] - 67s 1s/step - loss: 0.4481 - accuracy: 0.8482 - val_loss: 0.5925 - val_accuracy: 0.7479
Epoch 6/20
53/53 [==============================] - 69s 1s/step - loss: 0.4002 - accuracy: 0.8482 - val_loss: 0.6196 - val_accuracy: 0.7444
Epoch 7/20
53/53 [==============================] - 66s 1s/step - loss: 0.3878 - accuracy: 0.8553 - val_loss: 0.4678 - val_accuracy: 0.8071
Epoch 8/20
53/53 [==

In [ ]:
#save the model
model.save("vgg19_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# load the model
from tensorflow.keras.models import load_model
vgg19_model= load_model('vgg19_model.h5')

## Testing the model with random images

In [ ]:
# testing the model with an image

from tensorflow.keras.preprocessing import image

img = image.load_img('/content/dataset/diabetic_retinopathy/10099_right.jpeg',target_size =(240,240))
y=image.img_to_array(img)
y = np.expand_dims(y,axis = 0)
pred =np.argmax(vgg19_model.predict(y))
op =['cataract','diabetic_retinopathy','glaucoma','normal']
op[pred]

1/1 [==============================] - 0s 28ms/step


'diabetic_retinopathy'

#RESNET50

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization,Dropout
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# preparing the training and validation data
training_data = keras.preprocessing.image_dataset_from_directory(
    '/content/dataset',
    batch_size = 70,
    image_size =(224,224),

    shuffle = True,
    seed =123,
    subset ='training',
    validation_split=0.15,
    )
validation_data =keras.preprocessing.image_dataset_from_directory(
    '/content/dataset',
    batch_size = 70,
    image_size =(224,224),

    shuffle = True,
    seed =123,
    validation_split =0.15,
    subset ='validation',
    )


Found 4217 files belonging to 4 classes.
Using 3585 files for training.
Found 4217 files belonging to 4 classes.
Using 632 files for validation.


## Model Building

In [ ]:
  resnet_model = Sequential()
  pretrained_model= ResNet50(include_top=False,
                    input_shape=(224,224,3),
                    pooling='avg',
                    weights='imagenet')
  for layer in pretrained_model.layers:
    layer.trainable=False
  resnet_model.add(pretrained_model)
  resnet_model.add(Flatten())
  resnet_model.add(BatchNormalization())
  resnet_model.add(Dense(512, activation='relu'))
  resnet_model.add(BatchNormalization())
  resnet_model.add(Dense(4, activation='softmax'))


In [ ]:
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_3 (Flatten)         (None, 2048)              0         
                                                                 
 batch_normalization_98 (Ba  (None, 2048)              8192      
 tchNormalization)                                               
                                                                 
 dense_5 (Dense)             (None, 512)               1049088   
                                                                 
 batch_normalization_99 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dense_6 (Dense)             (None, 4)                

In [ ]:
# model checkpoint
from tensorflow.keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint(
    'resnet50best_model.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

In [ ]:
resnet_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# Train the model
resnet_model.fit(training_data,
                 validation_data=validation_data,
                 epochs=50,
                 batch_size=70,
                 steps_per_epoch=len(training_data),
                 validation_steps=len(validation_data),
                 callbacks=[model_checkpoint]
)

Epoch 1/50
52/52 [==============================] - ETA: 0s - loss: 0.5430 - accuracy: 0.8273
Epoch 1: val_accuracy improved from -inf to 0.85759, saving model to resnet50best_model.h5
52/52 [==============================] - 33s 502ms/step - loss: 0.5430 - accuracy: 0.8273 - val_loss: 0.4017 - val_accuracy: 0.8576
Epoch 2/50
52/52 [==============================] - ETA: 0s - loss: 0.1778 - accuracy: 0.9347
Epoch 2: val_accuracy improved from 0.85759 to 0.89241, saving model to resnet50best_model.h5
52/52 [==============================] - 28s 474ms/step - loss: 0.1778 - accuracy: 0.9347 - val_loss: 0.2975 - val_accuracy: 0.8924
Epoch 3/50
52/52 [==============================] - ETA: 0s - loss: 0.1126 - accuracy: 0.9612
Epoch 3: val_accuracy improved from 0.89241 to 0.89715, saving model to resnet50best_model.h5
52/52 [==============================] - 27s 477ms/step - loss: 0.1126 - accuracy: 0.9612 - val_loss: 0.2775 - val_accuracy: 0.8972
Epoch 4/50
52/52 [=========================

In [ ]:
# load the model
from tensorflow.keras.models import load_model
resnet_loadedmodel= load_model('resnet50best_model.h5')

In [ ]:
# Define a function for prediction of the image
def predict(img):
  y=image.img_to_array(img)
  y = np.expand_dims(y,axis = 0)
  pred =np.argmax(resnet_loadedmodel.predict(y))
  op =['cataract','diabetic_retinopathy','glaucoma','normal']
  return op[pred]

## Testing the model with random images

In [ ]:
img1 = image.load_img('/content/dataset/glaucoma/1211_right.jpg',target_size =(300,300))
predict(img1)

1/1 [==============================] - 2s 2s/step


'glaucoma'

In [ ]:
img2 = image.load_img('/content/dataset/normal/2599_left.jpg',target_size =(300,300))
predict(img2)

1/1 [==============================] - 0s 25ms/step


'normal'

In [ ]:
img3 = image.load_img('/content/dataset/cataract/2127_left.jpg',target_size =(300,300))
predict(img3)

1/1 [==============================] - 0s 37ms/step


'cataract'

In [ ]:
img4 = image.load_img('/content/dataset/diabetic_retinopathy/1000_left.jpeg',target_size =(300,300))
predict(img4)

1/1 [==============================] - 0s 39ms/step


'diabetic_retinopathy'

In [ ]:
img5 = image.load_img('/content/dataset/normal/2780_left.jpg',target_size =(300,300))
predict(img5)

1/1 [==============================] - 0s 25ms/step


'normal'

#Inception V3

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow as tf


In [ ]:
# split the dataset into train and test
"""
!pip install split-folders
import splitfolders
splitfolders.ratio('/content/dataset',output='data',seed=1337,ratio=(0.8,0.2))

In [ ]:
# Define the image dimensions and batch size
image_size = (224,224)
batch_size = 64

## Data Preprocessing

In [ ]:
# Data Augmentation

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

valid_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
train_data = train_datagen.flow_from_directory(
    directory='/content/data/train',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
)

validation_data = valid_datagen.flow_from_directory(
    directory='/content/data/val',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
  )

Found 3372 images belonging to 4 classes.
Found 845 images belonging to 4 classes.


## Model Building

In [ ]:
inception_model = InceptionV3(input_shape=(224, 224, 3),
                        include_top=False,
                        weights='imagenet')

for layer in inception_model.layers:
    layer.trainable = False

# Add custom layers
inception_x= inception_model.output

x = Flatten()(inception_x)
x=BatchNormalization()(x)
x= Dense(4, activation='softmax')(x)  # 4 categories

# Create new model
inception_model = Model(inputs=inception_model.input, outputs=x)


87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
# Compile the model
inception_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy'],
    loss='categorical_crossentropy'
)

In [ ]:
inception_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

In [ ]:
# ModelCheckpoint
model_checkpoint = ModelCheckpoint(
    'inceptionv3_best_model.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

In [ ]:
# train the model
inception_model.fit(train_data,
          validation_data=validation_data,
          epochs=10,
          batch_size=100,
          steps_per_epoch=len(train_data),
          validation_steps=len(validation_data),
          #callbacks=[model_checkpoint]
        )


Epoch 1/10
53/53 [==============================] - 77s 1s/step - loss: 0.8368 - accuracy: 0.6948 - val_loss: 0.6310 - val_accuracy: 0.7515
Epoch 2/10
53/53 [==============================] - 64s 1s/step - loss: 0.5668 - accuracy: 0.7859 - val_loss: 0.5430 - val_accuracy: 0.8024
Epoch 3/10
53/53 [==============================] - 65s 1s/step - loss: 0.4724 - accuracy: 0.8197 - val_loss: 0.5037 - val_accuracy: 0.8071
Epoch 4/10
53/53 [==============================] - 66s 1s/step - loss: 0.4170 - accuracy: 0.8455 - val_loss: 0.4709 - val_accuracy: 0.8166
Epoch 5/10
53/53 [==============================] - 71s 1s/step - loss: 0.3699 - accuracy: 0.8541 - val_loss: 0.4600 - val_accuracy: 0.8178
Epoch 6/10
53/53 [==============================] - 66s 1s/step - loss: 0.3453 - accuracy: 0.8713 - val_loss: 0.4737 - val_accuracy: 0.8367
Epoch 7/10
53/53 [==============================] - 65s 1s/step - loss: 0.3120 - accuracy: 0.8894 - val_loss: 0.4782 - val_accuracy: 0.8331
Epoch 8/10
53/53 [==

In [ ]:
# save the model
inception_model.save('incepion_v3_model.h5')

In [ ]:
# load the saved model
from tensorflow.keras.models import load_model
incepion_loaded_model= load_model('incepion_v3_model.h5')

In [ ]:
# create a function for prediction of image
def predict(img):
  y=image.img_to_array(img)
  y = np.expand_dims(y,axis = 0)
  pred =np.argmax(incepion_loaded_model.predict(y))
  op =['cataract','diabetic_retinopathy','glaucoma','normal']
  return op[pred]

## Testing the model with random images

In [ ]:
img1 = image.load_img('/content/dataset/normal/2780_left.jpg',target_size =(224,224))
predict(img1)

1/1 [==============================] - 2s 2s/step


'normal'

In [ ]:
img2 = image.load_img('/content/dataset/diabetic_retinopathy/1000_left.jpeg',target_size =(224,224))
predict(img2)

1/1 [==============================] - 0s 27ms/step


'diabetic_retinopathy'